# - Import modules -

In [1]:
import os
import time
import numpy as np
import random
from matplotlib import pyplot as plt
from math import exp, cos
import json
from tqdm import tqdm
import matplotlib.cm as cm
import scipy.stats as stats
import matplotlib
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
plt.rcParams["axes.labelweight"] = "bold"

# - Define functions -

In [2]:
## will create the files needed to store values in CWD/Data
def data_dir(dir="os.getcwd()"):            
    if not os.path.exists('{}/{}'.format(os.getcwd(),'DataLIF')): # used to tag simulation results
        directory = os.path.dirname(dir)
        os.makedirs('{}/{}'.format(os.getcwd(),'DataLIF'))
    if not os.path.exists('DataLIF/{}'.format(timestr)):
        os.makedirs('DataLIF/{}'.format(timestr))

In [3]:
## function to get the index "-delay/dt" in the list representing m(t-Delta)
def mdelta(synapse_value, delay, t, dt,i):
    if t < delay:  # if the delay is the time is too small, we take m(t-delta)=0
        return 0
    else:  # if the time is above or equal to the delay, we can find the m(t-delta)
        return synapse_value[int(-delay / dt)][i]

# - LIF parameters -

In [4]:
## setup parameters and state variables
duration       = 1000                  # total time (msec)
dt      = 0.100                      # time step (msec)
time    = np.arange(0, duration+dt, dt) # time array
t_rest  = 1                          # initial refractory time

In [5]:
## LIF properties
Vm      = np.zeros(len(time))-67    # potential (V) trace over time 
Rm      = 200000                   # resistance (kOhm)
#Cm      = 20                  # capacitance (uF)
tau_m   = 20 #Rm*Cm               # time constant (msec)
tau_ref = 1                   # refractory period (msec)
Vth     = -42                   # spike threshold (V)
V_spike = 100                 # spike delta (V)

In [6]:
## First Input stimulus
minI = 0.0      # beggining time (ms)
maxI = 100.0     # End input (ms)
Input = 0.0   # input current (A)

In [7]:
## Second Input stimulus
minI2 = 500.0      # beggining time (ms)
maxI2 = 600.0     # End input (ms)
Input2 = 0.0   # input current (A)

In [8]:
## Set Input for each step
Il = np.zeros(len(time))
#Il = np.random.normal(10,5,len(time)) # for random/noisy inputs

Il[int(minI/dt):int(maxI/dt)] = Input
Il[int(minI2/dt):int(maxI2/dt)] = Input2

In [9]:
## Set noise
N = 0.5

In [10]:
H = 0.05

In [11]:
time_value = np.arange(0, duration, dt) # creat the timescale, depending on the duration and step dt, for plotting

# - initial parameters - 

In [12]:
## Total time duration (ms)
duration = 1000   

## Default Time resolution (ms)
dt = 0.1

## Period occuring 
n_period = int(duration / dt) + 1

In [ ]:
## Initialization of the random generator (reproductibility)
w=np.random.randint(1,10)
np.random.seed(w)

In [13]:
 ## Weights (connextions pre->post ; notation: PostPre):
J = 1

 ## Tau:
tau_m = 20 #(ms)

 ## Delays:
D = 1 #(ms)

In [14]:
T = dict()

In [15]:
## inputs array generation: noises picked (N=sigma)
for stc in structures:
    
    T[stc] = np.random.normal(0, TN[stc] , size=n)
    
    I[stc] = np.random.normal(0, N[stc], size=(n_period, n)) + H[stc] + T[stc]

NameError: name 'structures' is not defined

# - Simulation Core -

In [ ]:
## iterate over each time step
it=0
nspike=np.zeros(int(duration/dt)+1)
for i, t in enumerate(time):
    
    if t > t_rest:
        Vm[i] = Vm[i-1] + Il[it]*Rm / tau_m * dt + Il[it]*Rm / tau_m * dt**2 + np.random.normal(0, N)
        if Vm[i]>=Vth:
            nspike[it] = 1
            #print('spike!',t)
    
    if Vm[i] >= Vth:
        Vm[i] += V_spike
        t_rest = t + tau_ref
    
    

    
    
    
    
    
    
    
    it+=1

In [ ]:
for t in tqdm(range(1, n_period)):    ## Update of Inputs
    
    for stc in structures:
        
        for i in range(n):
            
            con = input_mapping[stc] #multiplication by the probability of connexion below with n
            
            I[stc][t][i] += - np.sum(
                G[con]* mdelta(m[con][:t + 1], D[con], time_value[t], dt,i) *J[con][i]
                )/(np.sum(J[con][i]))  

                
    for con in connections:          ## Update of activities
        
        for i in range(n):
            
            stc = activities_mapping[con]
            
            dm[con][t][i] = dt * (- m[con][t][i] + Ic(I[stc][t][i])) / tau[con]
            
        if t < n_period-1: # we can't add last value of dm to the last value of m
            
            m[con][t+1]=m[con][t]+dm[con][t]
            
        else:
            
            continue

# - Firing rate calculation (window) - 

In [ ]:
## rate calculation
ratio=np.zeros(len(nspike))
for x in enumerate(nspike):
    #print(x)
    if x[0] <= 10 :
        y = np.sum(nspike[:5])/5
        #print(y)
    elif x[0] > 10 :
        y = np.sum(nspike[x[0]:x[0]+500])/1000
    ratio[x[0]]=y

# - Savedata - 

In [ ]:
timestr = time.strftime("%Y-%m-%d-%H:%M:%S") # give the date and time used to tag simulation results
data_dir()  # creating the 'Data' directory on CWD and the directory (with date & time) to store the results

np.save('DataLIF/{}/{}_Inputs'.format(timestr, stc), I[stc]) # save the Input array (npy format) for each neuron population in "structures"
    
np.save('DataLIF/{}/{}_Activities'.format(timestr, con), m[con]) # save the Activity array (npy format) for each neuron population in "connections"

datax={'Weight':G,'Delay':D,'tau':tau,'Noise':N,  # make one dict with all parameters used 
       'time_trial':t,'duration':duration,'step':dt}

with open('DataLIF/{}/0_Parameters.json'.format(timestr),'w') as f: # save the parameters used during the simulation
            json.dump(datax, f) 

# - Plotting- 

In [ ]:
## Membrane potential and Input trace
fig = plt.figure(figsize=(15,10))

ax1 = plt.subplot(311)
ax1.set_ylim(0,2)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_ylabel('Input [A]',fontsize=15,fontweight='bold')
plt.plot(time,Il,'b',linewidth=2)

    # share x only
ax2 = plt.subplot(312, sharex=ax1)
ax2.set_ylabel('M.potential [V]',fontsize=15,fontweight='bold')
ax2.set_xlabel('Time [ms]',fontsize=15,fontweight='bold')
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.plot(time, Vm,'g',linewidth=2)
ax2.plot((time[0], duration), (Vth, Vth), 'r',linestyle=':', dashes=(1, 5)) # put the treshold level
plt.setp(ax1.get_xticklabels(), visible=False)

# make these tick labels invisible
plt.setp(ax1.get_xticklabels(), visible=False)

#ax3 = plt.subplot(313, sharex=ax1)
#ax3.set_ylim(2,5)
#ax3.set_ylabel('Firing rate [Hz]',fontsize=15,fontweight='bold')
#ax3.set_xlabel('Time [ms]',fontsize=15,fontweight='bold')
#plt.plot(time, ratio,'r',linewidth=2)

plt.autoscale(enable=True, axis='x', tight=True)
plt.show()